<a href="https://colab.research.google.com/github/tnfru/colab_research/blob/master/PricePred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from tensorflow.nn import softmax

In [0]:
data = pd.read_excel('https://filebin.net/d71dl6ly0ucp7zwg/Beispiel_-_Superstore.xls?t=ak0pv1r2')

In [0]:
df = data.drop(['Kundenname'], axis=1)

In [0]:
df.dtypes, df.isnull().sum(), df.isna().sum()

In [0]:
def encode(df, column):
  df[column] = df[column].astype('category')
  df[column + "_cat"] = df[column].cat.codes

In [0]:
def encode_labels(df):
  labels_to_encode = ["Auftrags-ID", "Kunden-ID", "Bestelldatum", "Produkt-ID"]
  df = df.sort_values(by=["Bestelldatum"])

  for label in labels_to_encode:
    encode(df, label)

  labels_to_encode = ["Land/Region", "Bundesland", "Stadt"]
  df = df.sort_values(by=labels_to_encode)

  for label in labels_to_encode:
    encode(df, label)

  df = df.drop(["Auftrags-ID", "Kunden-ID", "Bestelldatum",
             "Land/Region", "Bundesland", "Stadt", "Versanddatum", 
             "Produkt-ID", "Zeilen-ID"], axis=1)
  return df

In [0]:
def get_separated_entities(df):
  df = encode_labels(df)

  items = df.drop(columns=["Kunden-ID_cat", "Bestelldatum_cat", "Auftrags-ID_cat", 
                "Land/Region_cat", "Bundesland_cat", "Stadt_cat", "Region", 
                "Segment", "Versandmodus"], axis=1)
  items = pd.get_dummies(items, columns=["Kategorie" , "Unterkategorie"])
  


  transactions = pd.get_dummies(df, columns=["Segment", "Versandmodus", "Region", "Kategorie"]).drop("Produktname", axis=1)

  customers = df.drop(columns=["Versandmodus", "Produktname", "Kategorie", "Gewinn",
                             "Bestelldatum_cat", "Produkt-ID_cat", "Kunden-ID_cat", "Unterkategorie"], axis=1)
  customers = pd.get_dummies(customers, columns=["Segment", "Region"])

  return items, customers, transactions
  

In [0]:
def cosine_sim(a, b):
  return a @ b / (np.linalg.norm(a) * np.linalg.norm(b))

In [0]:
def create_CPM(data):
  cpm_shape = data["Kunden-ID"].nunique(), data["Produkt-ID"].nunique()
  CPM = np.zeros(cpm_shape)
  cpm_inp = data.drop(columns=["Auftrags-ID", "Zeilen-ID", "Versanddatum", "Bestelldatum", 
                   "Versandmodus", "Kundenname", "Segment", "Stadt", "Bundesland", 
                   "Land/Region", "Region", "Kategorie", "Unterkategorie", "Umsatz",
                   "Menge", "Rabatt", "Gewinn", "Produktname"], axis=1)
  encode(cpm_inp, "Kunden-ID")
  encode(cpm_inp, "Produkt-ID")
  cpm_inp = cpm_inp.drop(["Kunden-ID", "Produkt-ID"], axis=1).values

  for i in range(len(cpm_inp)):
    cust_id = cpm_inp[i][0]
    prod_id = cpm_inp[i][1]

    CPM[cust_id][prod_id] = 1
  
  return CPM

In [0]:
def create_PPM(items, print_progress=False):
  ppm_shape = items["Produkt-ID_cat"].nunique(), items["Produkt-ID_cat"].nunique()
  PPM = np.zeros(ppm_shape)
  
  ppm_inp = items.drop(["Produktname", "Gewinn", "Rabatt"], axis=1)
  ppm_inp["Umsatz"] = ppm_inp["Umsatz"].values / ppm_inp["Menge"].values
  ppm_inp = ppm_inp.rename(columns={"Umsatz" : "Preis"}).drop(["Menge"], axis=1)
  ppm_inp['Preis'] = (ppm_inp['Preis'].values - ppm_inp['Preis'].values.mean()) / ppm_inp['Preis'].values.std()

  ppm_inp = ppm_inp.groupby(["Produkt-ID_cat"]).sum()
  number_of_entries = ppm_inp[["Kategorie_Bürobedarf", "Kategorie_Möbel", "Kategorie_Technik"]].values.sum(axis=1)
  ppm_inp = ppm_inp.divide(number_of_entries, axis=0)
  ppm_inp = ppm_inp.values

  n = ppm_shape[0]

  for i in range(n):
    for j in range(i + 1, n):
      PPM[i][j] = cosine_sim(ppm_inp[i], ppm_inp[j])

    if print_progress and i % 100 == 0:
      print(round(i / n, 2) * 100 , "% done")

  return PPM

In [0]:
def ppm_recommendations(PPM):
  return argmax(PPM, axis=1)

In [0]:
items, customers, transactions = get_separated_entities(df)

In [0]:
CPM = create_CPM(data)

In [0]:
PPM = create_PPM(items)

In [0]:
def get_item_prior(CPM, epsilon=1e-3):
  num_customers = CPM.shape[1]
  return (CPM.sum(axis=0) + epsilon) / (num_customers + 2*epsilon) #w'keit dass item von beliebigem nutzer gekauft wird

In [0]:
def get_user_prior(CPM, epsilon=1e-3):
  num_items = CPM.shape[0]
  return  (CPM.sum(axis=1) + epsilon) / (num_items + 2*epsilon) #w'keit dass nutzer ein beliebiges item kauft

In [0]:
def item_based_between_two(CPM, v, u, k=1, epsilon=1e-3): #eq7
  r_u = CPM[u,:]
  r_v = CPM[v,:]

  return (r_u[r_v == k].sum() + epsilon) / (r_u.sum() + 2*epsilon)

In [0]:
def item_based(CPM, u, i):
  prob = get_user_prior(CPM)[u]
  user_quantity = CPM.shape[0]

  for v in range(user_quantity):
    prob *= item_based_between_two(CPM, v, u, k=CPM[v][i])
  return prob ** (1/ (1 + user_quantity))

In [0]:
def user_based_between_two(CPM, j, i, k=1, epsilon=1e-3):
  return (CPM[CPM[:,j] == k][:,i].sum() + epsilon) / (CPM[:,i].sum() + 2*epsilon)

In [0]:
def user_based(CPM, u, i):
  prob = get_item_prior(CPM)[i]
  item_quantity = CPM.shape[1]

  for j in range(item_quantity):
    prob *= user_based_between_two(CPM, j, i, k=CPM[u][j])
  
  return prob ** (1/ (1 + item_quantity))

In [0]:
def hybrid_score(CPM, u, i):
  return user_based(CPM, u, i) * item_based(CPM, u, i)

In [188]:
hybrid_score(CPM, 10, 20)

0.848742963537693